In [13]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, dash_table, ctx
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import altair as alt
import json

df = pd.read_csv("~/Library/CloudStorage/OneDrive-UBC/HIV-Dash/data/raw/HIV.csv", low_memory=False)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40540 entries, 0 to 40539
Data columns (total 22 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   DATAFLOW                                                                        40540 non-null  object 
 1   REF_AREA:Geographic area                                                        40540 non-null  object 
 2   INDICATOR:Indicator                                                             40540 non-null  object 
 3   SEX:Sex                                                                         40540 non-null  object 
 4   TIME_PERIOD:Time period                                                         40540 non-null  object 
 5   OBS_VALUE:Observation Value                                                     40540 non-null  object 
 6   UNIT_MULTIPLIE

In [15]:
# Step 1: Pivot the dataframe to have each indicator on seperate column and remove the string before the column symbol on each column name
df_new = df.pivot_table(values='OBS_VALUE:Observation Value', index=['REF_AREA:Geographic area', 'SEX:Sex', 'TIME_PERIOD:Time period'], columns='INDICATOR:Indicator', dropna=True, aggfunc='first')
df_new.reset_index(inplace=True)
df_new.columns = df_new.columns.str.split(':').str[-1].str.strip()
df_new['Geographic area'] = df_new['Geographic area'].str.split(':').str[-1].str.strip()
df_new['Sex'] = df_new['Sex'].str.split(':').str[-1].str.strip()

# Since we are only going to look at the full picture of each country in terms of gender, we will only keep the rows where the value in the Sex column equals to Total
df_new = df_new[df_new['Sex'] == 'Total']

# Step 2: Aggregate the data for each 'Geographic area', 'Sex', and 'Time period'
# We will take the first non-null value for each group
aggregation_functions = {col: 'first' for col in df_new.columns[3:]}
df_aggregated = df_new.groupby(['Geographic area', 'Sex', 'Time period'], as_index=False).agg(aggregation_functions)

# Drop any remaining rows with all NaN values in the indicator columns
df_aggregated.dropna(subset=df_aggregated.columns[3:], how='all', inplace=True)

# Reset index to tidy up the DataFrame
df_aggregated.reset_index(drop=True, inplace=True)

df_aggregated

INDICATOR:Indicator,Geographic area,Sex,Time period,"Estimated rate of annual AIDS-related deaths (per 100,000 population)","Estimated incidence rate (new HIV infection per 1,000 uninfected population)",Reported number of children (aged 0-14 years) receiving antiretroviral treatment (ART),Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth,Reported number of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth,Estimated number of children (aged 0-17 years) who have lost one or both parents due to all causes,Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS,Per cent of pregnant women living with HIV receiving lifelong ART,Reported number of pregnant women living with HIV receiving lifelong antiretroviral treatment (ART),Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine),Reported number of pregnant woment living with HIV receiving anitretroviral treatments (ARVs) for prevention of mother to child transmission programmes (PMTCT),Mother-to-child HIV transmission rate,Per cent of young people (aged 15-24 years) who had more than one sexual partner in the past 12 months reporting the use of a condom during their last sexual intercourse,"Per cent of young people (aged 15-24 years) with comprehensive, correct knowledge of HIV",Per cent of young people (aged 15-24 years) who know a place to get tested for HIV,Per cent of people (aged 15-49 years) expressing discriminatory attitudes towards people living with HIV,Per cent of young people (aged 15-24 years) who have ever been tested for HIV and received the result of the last test
0,Afghanistan,Total,2000,0.04,0.02,0,None,None,1100000,2000,None,None,None,None,52.70,None,None,None,None,None
1,Afghanistan,Total,2001,0.04,0.02,0,None,None,1120000,2300,None,None,None,None,52.34,None,None,None,None,None
2,Afghanistan,Total,2002,0.04,0.02,0,None,None,1150000,2700,None,None,None,None,53.31,None,None,None,None,None
3,Afghanistan,Total,2003,0.04,0.02,0,None,None,1170000,3100,None,None,None,None,52.12,None,None,None,None,None
4,Afghanistan,Total,2004,0.08,0.02,0,None,None,1200000,3600,None,None,None,None,51.16,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3705,Zimbabwe,Total,2018,79.94,2.23,52400,62.0,40000,960000,660000,92.5,59600,92.5,59600,8.15,None,None,None,None,None
3706,Zimbabwe,Total,2019,76.39,1.82,59800,57.6,35400,920000,620000,93.7,57600,93.7,57600,7.57,None,None,None,None,None
3707,Zimbabwe,Total,2020,73.06,1.53,56400,76.5,44600,890000,570000,87.9,51200,87.9,51200,7.90,None,None,None,None,None
3708,Zimbabwe,Total,2021,68.16,1.33,52400,>95,54300,860000,530000,85.4,47000,85.4,47000,8.42,None,None,None,None,None


Check if there are any empty columns 

In [16]:
df_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3710 entries, 0 to 3709
Data columns (total 20 columns):
 #   Column                                                                                                                                                                     Non-Null Count  Dtype 
---  ------                                                                                                                                                                     --------------  ----- 
 0   Geographic area                                                                                                                                                            3710 non-null   object
 1   Sex                                                                                                                                                                        3710 non-null   object
 2   Time period                                                                                 

Since column 15-19 are almost fully empty, we will remove them from the dataframe 

In [17]:
df_aggregated = df_aggregated.iloc[:, 0:15]

Update the data types for variables

In [18]:
## Convert time period from string to datetime
df_aggregated['Time period'] = pd.to_datetime(df_aggregated['Time period'], format='%Y-%m-%d').dt.year

## Convert fully numeric indicators into numeric type
def is_numeric(column):
    try:
        pd.to_numeric(column)
        return True
    except ValueError:
        return False
numeric_columns = df_aggregated.iloc[:, 3:15].apply(is_numeric)
numeric_column_names = numeric_columns[numeric_columns].index
df_aggregated[numeric_column_names] = df_aggregated[numeric_column_names].apply(pd.to_numeric)

/var/folders/37/64df_33d13jdmmdy_f16_cd40000gn/T/ipykernel_11608/2197229970.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/37/64df_33d13jdmmdy_f16_cd40000gn/T/ipykernel_11608/2197229970.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Check which indicator columns are still in object data type

In [19]:
df_aggregated.iloc[:, 3:15].select_dtypes(include=['object']).columns

Index(['Estimated rate of annual AIDS-related deaths (per 100,000 population)',
       'Estimated incidence rate (new HIV infection per 1,000 uninfected population)',
       'Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth',
       'Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS',
       'Per cent of pregnant women living with HIV receiving lifelong ART',
       'Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine)',
       'Mother-to-child HIV transmission rate'],
      dtype='object', name='INDICATOR:Indicator')

Find out the string values in each indicator columns that are still in object data type

In [20]:
object_columns = df_aggregated.iloc[:, 3:15].select_dtypes(include=['object']).columns

for col in object_columns:
    non_numeric_values = []

    try:
        # Try to convert the column to numeric
        pd.to_numeric(df_aggregated[col])
    except ValueError as e:
        # Capture the non-numeric values that caused the error
        non_numeric_values = df_aggregated[col][~df_aggregated[col].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()]

    if non_numeric_values.any():
        print(f"Column '{col}' has non-numeric values:")
        print(f"{non_numeric_values.unique()}\n")

Column 'Estimated rate of annual AIDS-related deaths (per 100,000 population)' has non-numeric values:
['<0.01' None]

Column 'Estimated incidence rate (new HIV infection per 1,000 uninfected population)' has non-numeric values:
['<0.01' None]

Column 'Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth' has non-numeric values:
[None '<1' '>95']

Column 'Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS' has non-numeric values:
['<100' '<200' '<500' None]

Column 'Per cent of pregnant women living with HIV receiving lifelong ART' has non-numeric values:
[None '<1' '>95']

Column 'Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine)' has non-numeric values:
[None '<1' '>95']

Column 'Mother-to-child HIV transmission rate' has non-numeric values:
[None '<0.01']



Transform the string values into numeric value in the non-numeric indicator columns through substitution with a reasonable assumption\
\- For any of the string value that shows the less than(<) or greater than(>) sign, transform them into the mean value of the lower and upper bound of the less than or greater than value.  

In [21]:
columns_to_replace = [
    'Estimated rate of annual AIDS-related deaths (per 100,000 population)',
    'Estimated incidence rate (new HIV infection per 1,000 uninfected population)',
    'Mother-to-child HIV transmission rate',
    'Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth',
    'Per cent of pregnant women living with HIV receiving lifelong ART',
    'Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine)',
    'Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS'
]

values_to_replace = {
    '<0.01': '0.005',
    '<1': '0.5',
    '>95': '97.5',
    '<100': '50',
    '<200': '150',
    '<500': '250'
}

df_aggregated[columns_to_replace] = df_aggregated[columns_to_replace].replace(values_to_replace)

df_aggregated[columns_to_replace] = df_aggregated[columns_to_replace].apply(pd.to_numeric, errors='coerce')

/var/folders/37/64df_33d13jdmmdy_f16_cd40000gn/T/ipykernel_11608/2534955969.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/37/64df_33d13jdmmdy_f16_cd40000gn/T/ipykernel_11608/2534955969.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Verify if all indicator columns are in numeric data type

In [22]:
df_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3710 entries, 0 to 3709
Data columns (total 15 columns):
 #   Column                                                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                                                           --------------  -----  
 0   Geographic area                                                                                                                                                  3710 non-null   object 
 1   Sex                                                                                                                                                              3710 non-null   object 
 2   Time period                                                                                                                     

In [ ]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output, dash_table, State, clientside_callback, ClientsideFunction
import dash_bootstrap_components as dbc
import dash_mantine_components as dmc
from dash_iconify import DashIconify
import plotly.express as px
import plotly.graph_objects as go
import altair as alt
import json

df_aggregated = pd.read_csv("data/processed/dash_clean.csv", index_col=0, low_memory=False)

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

## Add dataset and github icon
def modal_data_source():
    return dmc.Modal(
            id='modal-data-source',
            size='55%',
            styles={
                'modal': {
                    'background-color': '#f2f2f2',
                }
            },
            children=[
                dcc.Markdown(
                    [
                        """
                        
                        # About the Dataset
                        
                        HIV (Human Immunodeficiency Virus) is a well-known virus that attacks the immune system of infected individuals. 
                        HIV gained immense awareness in the early 1980s, particularly in the USA, and it is responsible for AIDS (Acquired Immunodeficiency Syndrome), 
                        which is a stage of HIV infection where the immune system is severely damaged. As of 2022, 
                        there were approximately 39 million people globally that are living with HIV/AIDS, which has no known cure so far. 
                        HIV/AIDS remains a global public health issue, with ongoing efforts to increase access to prevention, treatment, and care.

                        This dataset contains 12 indicators. These indicators can be categorized into two main groups:

                        1. General rates (ie. rate of annual AIDS-related deaths, incidence rate, mother-to-child transmission rate)
                        2. Statistics on pregnant women and children (ie. number of children receiving ART, percent of pregnant women living with HIV within 2 months of birth, 
                           estimated children who have lost one or both parents due to AIDS)

                        ## Source Information
                        
                        - **Title:** "Key HIV epidemiology indicators for children and adolescents aged 0-19, 2000-2022"
                        - **Published Online:** data.unicef.org
                        - **Retrieved From:** [UNICEF](https://data.unicef.org/resources/dataset/hiv-aids-statistical-tables/)
                        - **Temporal Coverage:** From 01/01/2000 to 12/31/2022
                        - **Geospatial Coverage:** Worldwide
                        
                        ## Collection Methodology
                        
                        The data was collected by visiting the publisher.
                        
                        """
                    ],
                )
            ]
        )

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(
            html.Div([
                html.H1("HIV Indicator Dashboard", style={"color": "burgundy"}),
                dmc.Grid(
                    [
                        modal_data_source(),
                        dmc.Col(
                            dmc.Group(
                                [
                                    dmc.ActionIcon(
                                        [DashIconify(icon='bx:data', color='#C8C8C8', width=25)],
                                        variant='transparent',
                                        id='about-data-source'
                                    ),
                                    dmc.Anchor(
                                        [DashIconify(icon='uil:github', color='#8d8d8d', width=30)],
                                        href='https://github.com/shaytran/HIV-Dash'
                                    )
                                ],
                                spacing='xl',
                                position='center'
                            ),
                        )
                    ],
                    justify="center"
                )
            ], style={"textAlign": "center"}),
            width={"size": 12, "offset": 0}
        )
    ], align="center", style={"margin-bottom": "20px"}),
    html.Img(src="https://cdn.storymd.com/optimized/RqVLDEsxom/thumbnail.gif", style={"display": "block", "margin-left": "auto", "margin-right": "auto", "width": "10%"}),
    html.P("HIV (Human Immunodeficiency Virus) attacks the immune system and can lead to AIDS (Acquired Immunodeficiency Syndrome), a condition where the immune system is severely damaged. First identified in the early 1980s in the USA, HIV/AIDS has since become a global public health issue, with approximately 39 million people living with the virus worldwide as of 2022. Despite significant advancements, there is no cure for HIV/AIDS, highlighting the importance of continued efforts in prevention, treatment, and care.", style={"textAlign": "center", "color": "burgundy"}),
    dcc.Tabs(id='tabs', children=[
        dcc.Tab(label='Indicator Trend', children=[
            html.H2('HIV Indicator Trends by Country and Year', style={"textAlign": "center"}),
            html.P('Select an HIV indicator and up to 4 countries to compare their trends over time. You can toggle the year range using the sliding bar.', style={"textAlign": "center"}),
            html.Div(id='missing-data-warning', style={'textAlign': 'center', 'color': 'red'}),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='indicator-dropdown',
                        value=df_aggregated.columns[3],  # Default value is the first indicator column
                        options=[{'label': col, 'value': col} for col in df_aggregated.columns[3:]], 
                        placeholder='Choose 1 indicator...'
                    ),
                    dcc.Dropdown(
                        id='country-dropdown',
                        options=[{'label': country, 'value': country} for country in df_aggregated['Geographic area'].unique()],
                        placeholder='Choose up to 4 countries...',
                        multi=True
                    ),
                    html.Div([
                        dcc.RangeSlider(
                            id='year-slider',
                            min=df_aggregated['Time period'].min(),
                            max=df_aggregated['Time period'].max(),
                            marks={str(year): str(year) for year in range(df_aggregated['Time period'].min(), df_aggregated['Time period'].max() + 1)},
                            step=1,
                            value=[df_aggregated['Time period'].min(), df_aggregated['Time period'].max()]
                        )
                    ], style={'marginTop': 20})
                ])
            ]),
            html.Div(id='trend-chart')
        ]),
        ### Second tab
        dcc.Tab([
            html.H2("Global Spread of HIV Indicator", style={"textAlign": "center"}),
            dbc.Row([
                # Dropdown for selecting indicator
                dcc.Dropdown(
                    id='indicator-map-dropdown',
                    # Limiting the usage of indicator columns to avoid errors since some of them are in non-numeric type at the moment
                    options=[{'label': col, 'value': col} for col in df_aggregated.columns[3:]],
                    value=df_aggregated.columns[3],  # Initial indicator
                    multi=False,
                )]),
            dbc.Row([
                # Map
                dbc.Col(dcc.Graph(id='world-map'))
            ])   
        ], label='Indicator Map'),
        ### Third tab
        dcc.Tab([
            html.H2('Indicator Summary Statistics'),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='indicator-stats-dropdown',
                        value=df_aggregated.columns[4],  # Default value is the first indicator column
                        options=[{'label': col, 'value': col} for col in df_aggregated.columns[4:]], 
                        placeholder='Choose 1 indicator...'
                    ),
                    dcc.Dropdown(
                        id='country-stats-dropdown',
                        options=[{'label': country, 'value': country} for country in df_aggregated['Geographic area'].unique()],
                        placeholder='Choose up to 10 countries...',
                        multi=True  # Allow multiple selections, but limit to 2 for the stats comparison
                    )
                ]),
                dcc.RangeSlider(
                    id='year-stats-slider',
                    min=df_aggregated['Time period'].min(),
                    max=df_aggregated['Time period'].max(),
                    value=[df_aggregated['Time period'].min(), df_aggregated['Time period'].max()],
                    marks={str(year): str(year) for year in range(df_aggregated['Time period'].min(), df_aggregated['Time period'].max() + 1)},
                    step=1,
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ]),
            dbc.Row([
                dash_table.DataTable(id='summary-stats-table')
            ])
        ], label='Indicator Summary Statistics')
    ])    
], style={'backgroundColor': '#FFFFFF', "color": "#2F3C48"})

## Callback for the dataset and github icons
clientside_callback(
    ClientsideFunction(namespace='clientside', function_name='toggle_modal_data_source'),
    Output('modal-data-source', 'opened'),
    Input('about-data-source', 'n_clicks'),
    State('modal-data-source', 'opened'),
    prevent_initial_call=True
)

# Callback for updating the chart based on selections
@app.callback(
    [Output('trend-chart', 'children'), Output('missing-data-warning', 'children')],
    [Input('indicator-dropdown', 'value'),
     Input('country-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_chart(selected_indicator, selected_countries, selected_years):
    if selected_countries is None or selected_indicator is None or len(selected_countries) > 4:
        return [html.Div('Please select an indicator and up to 4 countries.')], None

    chart_df = df_aggregated[(df_aggregated['Time period'].between(selected_years[0], selected_years[1])) & (df_aggregated['Geographic area'].isin(selected_countries))]

    if chart_df.empty:
        return [html.Div()], 'No data available for the selected criteria.'

    missing_countries = [country for country in selected_countries if country not in chart_df['Geographic area'].unique()]
    warning_msg = None
    if missing_countries:
        warning_msg = f"Missing data for countries: {', '.join(missing_countries)}"

    # Creating a line chart using Plotly
    fig = px.line(chart_df, x="Time period", y=selected_indicator, color='Geographic area', title=f"Trend of {selected_indicator}", labels={"Time period": "Year"})
    
    # Update the layout to remove y-axis title
    fig.update_layout(
        yaxis_title="",
        margin=dict(l=100, r=100, t=100, b=100),  
        width=1200,  
        height=600,  
        transition_duration=500
    )


    return [dcc.Graph(figure=fig)], warning_msg

# Callback for updating the map based on dropdown and slider values
@app.callback(
    Output('world-map', 'figure'),
    Input('indicator-map-dropdown', 'value')
)

def update_figures(selected_indicator):
    fig_map = px.scatter_geo(
        df_aggregated,
        locations='Geographic area',
        locationmode='country names',
        color=selected_indicator,
        color_continuous_scale=px.colors.cyclical.IceFire,
        hover_name='Geographic area',
        # Limit the information to display on the hover box
        hover_data={'Time period':False,
                    'Geographic area':False,
                    'Estimated rate of annual AIDS-related deaths (per 100,000 population)':False,
                    'Estimated incidence rate (new HIV infection per 1,000 uninfected population)':False,
                    'Reported number of children (aged 0-14 years) receiving antiretroviral treatment (ART)':False,
                    'Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth':False,
                    'Reported number of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth':False,
                    'Estimated number of children (aged 0-17 years) who have lost one or both parents due to all causes':False,
                    'Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS':False,
                    'Per cent of pregnant women living with HIV receiving lifelong ART':False,
                    'Reported number of pregnant women living with HIV receiving lifelong antiretroviral treatment (ART)':False,
                    'Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine)':False,
                    'Reported number of pregnant woment living with HIV receiving anitretroviral treatments (ARVs) for prevention of mother to child transmission programmes (PMTCT)':False,
                    'Mother-to-child HIV transmission rate':False},
        labels={'size': 'Value',
                'Estimated rate of annual AIDS-related deaths (per 100,000 population)': "Estimated rate",
                'Estimated incidence rate (new HIV infection per 1,000 uninfected population)': "Estimated rate",
                'Reported number of children (aged 0-14 years) receiving antiretroviral treatment (ART)': "Reported number",
                'Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth': "Reported rate",
                'Reported number of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth': "Reported number",
                'Estimated number of children (aged 0-17 years) who have lost one or both parents due to all causes': "Estimated number",
                'Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS': "Estimated number",
                'Per cent of pregnant women living with HIV receiving lifelong ART': "Reported rate",
                'Reported number of pregnant women living with HIV receiving lifelong antiretroviral treatment (ART)': "Reported number",
                'Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine)': "Reported rate",
                'Reported number of pregnant woment living with HIV receiving anitretroviral treatments (ARVs) for prevention of mother to child transmission programmes (PMTCT)': "Reported number",
                'Mother-to-child HIV transmission rate': "Reported rate"
                },
        animation_frame='Time period',
        animation_group='Geographic area',
        size=df_aggregated[selected_indicator].fillna(0),
        size_max=20,
        projection='natural earth',
    )
    # Remove the legend since some of the column names are too long and will squeeze the size of the map
    # fig.update(layout_coloraxis_showscale=False)
    fig_map.update_geos(
        showcountries=True, countrycolor='whitesmoke',
        showland=True, landcolor='dimgrey',
        showocean=True, oceancolor="Black"
    )
    fig_map.update_layout(
        title=f"{selected_indicator}",
        title_x=0.5,
        title_font_size=14,
        height=650,
        width=1300
    )
    return fig_map

# Callback for updating the summary statistics table
@app.callback(
    [Output('summary-stats-table', 'data'), Output('summary-stats-table', 'columns')],
    [Input('indicator-stats-dropdown', 'value'),
     Input('country-stats-dropdown', 'value'),
     Input('year-stats-slider', 'value')]
)
def update_summary_statistics(selected_indicator, selected_countries, selected_years):
    print("Callback Triggered")  # For debugging, check the console where you run the server

    if not selected_countries or not selected_indicator or len(selected_countries) > 10:
        return [], []

    # Filter based on the selected years and countries
    stats_df = df_aggregated[df_aggregated['Time period'].between(*selected_years)]
    stats_df = stats_df[stats_df['Geographic area'].isin(selected_countries)]

    # Compute summary statistics
    summary = stats_df.groupby('Geographic area')[selected_indicator].agg(['mean', 'min', 'max', 'count']).reset_index()

    # Rename columns for better readability in the DataTable
    summary.columns = ['Geographic area', 'Mean', 'Min', 'Max', 'Non-null Count']

    # Convert to records format for DataTable
    data = summary.to_dict('records')
    columns = [{"name": i, "id": i} for i in summary.columns]

    return data, columns

if __name__ == '__main__':
    app.run_server(debug=True)